#  Portfolio 2


I am also learning about network flows in one of my other computer science courses right now so I thought it would be intersting to implement an algorithm to find the max flow and then compare how this algorithm works vs formatting the problem as a classic lp-problem

## Ford-Fulkerson Algorithm 
 Is an algorithm to find the max-flow of a network.
 The ford fulkerson algorithm is an iterative method to find the max flow of a graph that contains a source and a sink and has capacity constraints for every edge on the graph. Like every max-flow problem we have to have flow conservation at each of the nodes except for the source and sink nodes. In my description of the problem and algorithm below I will talk about the source node as `s` and the sink node as `t`. 

 The general idea of the algorithm is of we can find a path from s to t such that the capacity of all the edges along the path are  > 0 then we will push the maximum amount of flow from s to t such that the flow is less than or equal to the capacity for each edge along the path. This is where the idea of a residual network comes in. The residual network is the network that represents the capacities of the graph after we push some flow from s to t. So when we push a flow of f from s to t we will subtract f from the capacity of each of the edges that we pushed and we will add f in the reverse direction to each of the edges. To better understand this I will give a simple example. Image we have a graph with only s and t nodes and we have a capacity of 5 along the edge from s to t, the max-flow we can push will be 5, so the residual graph will the edge from s to t with a value of 0 and the edge from t to s with a value of 5. After we have found the residual network we will repeat the step of finding a path from s to t on the residual network and pushing the maximum amount of flow along the path. We will continue to repeat until we can no longer find a path from s to t and then we know we have reached our maximum flow. 

In [83]:
import numpy as np
import networkx as nx
from typing import Dict, List
import cvxpy as cp
import timeit

In our algorithm we have a need to find if there is a path from s to t so we can use  a version of breadth first search. 


Breadth first search implementation ideas:
For our version of BFS we not only want to find out if there is a path that exists from s to t but we also want to  make sure that all of the edges of our path have a capacity that is greater than 0. 
So how we will do this is we will start on our starting vertex we will then check it neighbors that have the proper edge weights if they do then they will be up next on the queue then we will take the first element in the queue after checking all adjancent vertices and do the same for that vertex adding every adjacent vertex to the queue if it follows the edge constraint, we will continue until we find the t vertex or all the vertices have been visited.

In [86]:
def bfs(graph: Dict[str, Dict[str, int]], s: str, t: str) -> List[str]:
    visited = []
    queue = [[s]]
    
    while queue:
        path = queue.pop(0)
        node = path[-1]
        if node == t:
            return path
        if node not in visited:
            visited.append(node)
            children = [child for child, edge_value in graph[node].items() if edge_value > 0]
            for child in children:
                new_path = list(path)
                new_path.append(child)
                queue.append(new_path)

    return []

In [16]:
example_graph = {
    'A': {'B': 10, 'C': 5},
    'B': {'D': 15, 'E': 10},
    'C': {'D': 5, 'E': 15},
    'D': {},
    'E': {},
    'F': {},  # 'F' has no outgoing edges
}

bfs(example_graph, 'A', 'E')

['A', 'B', 'E']

As described at the beginning we will now implement the ford-fulkerson algorithm.

We will use a recursive approach for this algorithm, first we will start by setting the max-flow to 0 as we have no flow when the graph starts.

In [87]:
from typing import Dict, List

def ford_fulkerson(graph: Dict[str, Dict[str, int]], source: str, sink: str) -> int:
    # initialize the flow to 0
    max_flow = 0
    #determine if there is a path from s to t
    path = bfs(graph, source, sink)
    if not path: 
        # if there is no such path that exists return the max_flow
        return  max_flow
    else:
        # set the path flow to be infinity this is just a large placeholder value
        path_flow = np.inf
        # for each edge along the path from s to t find the minimum capacity and set the path_flow to that value
        for i in range(len(path) -1):
            path_flow = min(path_flow, graph[path[i]][path[i+1]])
        
        # create the residual graph and add the path flow to the total flow
        for i in range(len(path) -1):
            graph[path[i]][path[i+1]] = graph[path[i]][path[i+1]] - path_flow 
            graph[path[i+1]][path[i]] = graph[path[i+1]][path[i]] + path_flow
        max_flow += path_flow
    # return the max_flow and add the flow which you can get from the residual netwrok. 
    return max_flow + ford_fulkerson(graph, source, sink )



In [88]:
graph = {
    's': {'a': 10, 'b': 5, 'c': 0, 'd': 0, 't': 0},
    'a': {'s': 0, 'b': 15, 'c': 10, 'd': 0, 't': 0},
    'b': {'s': 0, 'a': 0, 'c': 0, 'd': 5, 't': 0},
    'c': {'s': 0, 'a': 0, 'b': 0, 'd': 0, 't': 10},
    'd': {'s': 0, 'a': 0, 'b': 0, 'c': 5, 't': 10},
    't': {'s': 0, 'a': 0, 'b': 0, 'c': 0, 'd': 0}
}

bfs(graph, 's', 't')

['s', 'a', 'c', 't']

In [53]:

ford_fulkerson(graph, 's', 't')

15

In [103]:
ff = timeit.timeit(lambda: ford_fulkerson(graph, 's', 't'), number=1)
ff

1.172899646917358e-05

In [79]:
adjacency_matrix =  np.array([
    [0, 10, 5, 0, 0, 0],
    [0, 0, 15, 10, 0, 0],
    [0, 0, 0, 0, 5, 0],
    [0, 0, 0, 0, 0, 10],
    [0, 0, 0, 5, 0, 10],
    [0, 0, 0, 0, 0, 0]
])

adjacency_matrix.T

array([[ 0,  0,  0,  0,  0,  0],
       [10,  0,  0,  0,  0,  0],
       [ 5, 15,  0,  0,  0,  0],
       [ 0, 10,  0,  0,  5,  0],
       [ 0,  0,  5,  0,  0,  0],
       [ 0,  0,  0, 10, 10,  0]])

Now that we have our max-flow value for the graph above with the ford-fulkerson algorithm. Let's try to set up the problem as a classic linear programming problem. 

Our objective function will be to maximize the flow out of the source node s. We can write this as 

 maximize $ \sum_{v} f(s,v) - f(v,s) $
 subject to the constraints.

 - the flow into each node must equal the flow out. 
-  the flow through each edge must be less than the capacity

let' s define this as an lp problem and solve it using cvxpy
 

In [82]:
flow = cp.Variable((len(adjacency_matrix), len(adjacency_matrix)))
obj = cp.Maximize(cp.sum(flow[0]))
constraints = []
for i in range(len(adjacency_matrix)):
    for j in range(len(adjacency_matrix)):
        summed_column = 0
        constraints.append(flow[i][j] <= adjacency_matrix[i][j])
        summed_column += flow[j][i]
   

for i in range(1, len(adjacency_matrix)-2):
    summed_column = 0
    for j in range(1, len(adjacency_matrix) -2):
        summed_column += flow[j][i]
    summed_row = cp.sum(flow[i])
    constraints.append(summed_row == summed_column)

prob = cp.Problem(obj, constraints)
prob.solve()

/Users/elisegreen/opt/anaconda3/envs/dsci430/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


15.000000000418883

In [105]:
lp =timeit.timeit(lambda: prob.solve(), number = 1)
lp

0.003266074003477115

In [107]:
lp/ff

278.46150453374986

**Conclusion**
As we can see both ways of solving the max-flow problem give us equivalent solutions. I also was curious to see if there was much of a difference in the time it took for each algorithm to run so I used the time-it function to track the total time each function ran for and I found that the ford-fulkerson algorithm was a bit quicker as it was 1.17e-05 s for it to run vs the linear programming approach took 0.0033s to run so the lp took approximatly 280 times longer to run which is a fair amount longer. This does make some sense as we know that when we use BFS to find the paths in the ford-fulkerson the total runtime must be $O(V E^2)$ where V is the number of vertices and E is the number of edges. However, LP problems have an exponential bound on running time but do tend to converge much faster than that usually and in some cases where there may be a lot of edges in a graph the LP may be quicker. 

**What I learned**
In this learning portfolio I learned how to implement two different strategies for solving max-flow problems.  This took some tough thinking to implement the algorithms even though I was already familiar with the ford-fulkerson algorithm I found that actually implementing it was difficult but it helped to cement my understanding. I think that it was interesting to explore a problem that can very easily be formatted as a linear programming problem be formatted in a different type of algorithm.